In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
#device = "cpu"

if device == "cuda":
    print(f"CUDA available. Using device \"{torch.cuda.get_device_name()}\".")
else:
    print(f"Using CPU device.")

CUDA available. Using device "NVIDIA GeForce GTX 1650 with Max-Q Design".


In [2]:
### NUMERICAL SIMULATOR ###
import scipy.integrate
def numerical_schrodinger(initials, ts, grid_size=100, grid_length=1):

    psi0 = initials[0:2, :, :]    
    v = initials[2, :, :]
    shape = psi0.shape
    flattened_shape = np.prod(shape)
    
    # flatten
    psi0 = np.reshape(psi0, flattened_shape)
    
    # construct laplacian operator and then Hamiltonian
    dx = grid_length/grid_size
    D2 = -2*np.eye(grid_size)
    for i in range(grid_size-1):
        D2[i,i+1] = 1 
        D2[i+1,i] = 1
    
    KE = -0.5*D2/(dx**2)
 
    def dpsi_dt(t,y):        
        y = np.reshape(y, shape)
        psi_real = y[0]
        psi_imag = y[1]
        dpsi_real = np.expand_dims(-KE@psi_imag - v*psi_imag, 0)
        dpsi_imag = np.expand_dims(KE@psi_real + v*psi_real, 0)
        return np.reshape(np.concatenate((dpsi_real, dpsi_imag), axis=0), flattened_shape)
    
    #sol = scipy.integrate.odeint(dpsi_dt, psi0, ts)
    sol = scipy.integrate.solve_ivp(dpsi_dt, t_span=[0,np.max(ts)], y0=psi0, t_eval=ts, method="RK23")
    
    return np.reshape(sol.y, shape+(len(ts),))

In [3]:
### TRAINING DATA ###
from tqdm import tqdm
import scipy.interpolate

class SimpleFiniteStepDataset(torch.utils.data.Dataset):
    def __init__(self, grid_size=100, grid_length=1, fourier_modes=2, max_time=2.0, ntimes=50, num_initials=500):
        self.grid_size = grid_size
        self.grid_length = grid_length
        self.fourier_modes = fourier_modes
        self.max_time = max_time
        self.ntimes = ntimes
        self.num_data = num_initials*ntimes*grid_size
        initials = np.empty((3, grid_size, num_initials))
        
        xs = np.linspace(0,grid_length,grid_size)
        
        print('Generating Initials')
        for i in range(num_initials):
            psi0_real, psi0_imag, v = self._generate_initial()
            initials[0, :, i] = psi0_real.T
            initials[1, :, i] = psi0_imag.T
            initials[2, :, i] = v.T
        print('Finished generating initials.')
        
        ts = np.linspace(0, max_time, ntimes)
        integrated = numerical_schrodinger(initials, ts, grid_size, grid_length)
        
        self.data = []
        
        print('the end bit is slow. Starting it now.')
        for i in range(self.num_data):
            a = i%self.ntimes                 # time index
            b = int(i/self.ntimes)%grid_size  # space index
            c = int(i/self.ntimes/grid_size)  # psi0 index
            
            x_real = initials[0,:,c]
            x_imag = initials[1,:,c]
            x_potl = initials[2,:,c]
            
            x = np.concatenate((np.array([xs[b], ts[a]]), x_real, x_imag, x_potl))
            
            y_real = integrated[0,b,c,a]
            y_imag = integrated[1,b,c,a]
            
            y = np.array([y_real, y_imag])
            
            x = torch.tensor(x).float()
            y = torch.tensor(y).float()
            
            self.data.append([x,y])
            
        print('did the end bit')
        
    def __len__(self):
        return self.num_data
    
    def __getitem__(self, index):    
        return self.data[index]
                
    def _generate_initial(self):
        
        # create the initial wave function
        fourier_real_coefficients = 2*np.random.rand(self.fourier_modes)-1
        fourier_imag_coefficients = 2*np.random.rand(self.fourier_modes)-1
        n = np.arange(start=1, stop=self.fourier_modes+1, step=1)

        scale_factor = np.sum(fourier_real_coefficients**2) + np.sum(fourier_imag_coefficients**2)
        scale_factor = (2/(self.grid_length*scale_factor))**0.5
        fourier_real_coefficients *= scale_factor
        fourier_imag_coefficients *= scale_factor
        
        def init_wave_function(x):
            x = np.pi*x/self.grid_length
            psi_real = np.sin(np.outer(x, n))
            psi_real = psi_real*fourier_real_coefficients
            psi_real = np.sum(psi_real, axis=-1)
            
            psi_imag = np.sin(np.outer(x, n))
            psi_imag = psi_imag*fourier_imag_coefficients
            psi_imag = np.sum(psi_imag, axis=-1)
            
            return psi_real, psi_imag
        
        
        # TODO: change when we have a better potential
        potential_function = lambda x: 0*x
        
        x = np.linspace(0, self.grid_length, self.grid_size)
        psi_real, psi_imag = init_wave_function(x)
        v = potential_function(x)
        
        return psi_real, psi_imag, v
        
    

In [4]:
# Generate training data
import time

print('Generating data...')
start = time.perf_counter()
data = SimpleFiniteStepDataset(grid_size=100, fourier_modes=2, max_time=0.5, ntimes=50, num_initials=200)
end = time.perf_counter()
print(f'Training data generated after {end-start} seconds!')

Generating data...
Generating Initials
Finished generating initials.
the end bit is slow. Starting it now.
did the end bit
Training data generated after 35.76319729100214 seconds!


In [5]:
### MODEL DEFN ###
class SimpleFiniteStepSolver(nn.Module):
    def __init__(self, grid_size=100, hidden_dim=20):
        super(SimpleFiniteStepSolver, self).__init__()
        
        self.grid_size = grid_size

        self.mlp = nn.Sequential(
            nn.Linear(3*grid_size+2, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim, 2)
        )

    def forward(self, x):
        return self.mlp(x)

In [8]:
# Try to train
from tqdm import tqdm
import matplotlib.pyplot as plt # DEBUG TMP

nepochs = 100
model = SimpleFiniteStepSolver(grid_size=100, hidden_dim=300).to(device)

train_data_loader = torch.utils.data.DataLoader(data, batch_size=1000, shuffle=True)

optm = torch.optim.Adam(model.parameters(), lr = 0.001)

mse_hyperparam = 1
norm_hyperparam = 0
schrod_hyperparam = 0

# def norm_factor(output):
#     norm = torch.sum(output**2)* data.grid_length / (np.prod(output.shape) / 2)
#     norm_factor = (norm-1)**2
#     return norm_factor

grid_size = model.grid_size
def schrodinger_loss(output, in_put):
    batch_size = output.shape[0]
    
    psi_real = output[:,0]
    psi_imag = output[:,1]
    
    # Calculate Laplacian    
    e_x = torch.zeros((batch_size,302)).to(device)
    e_x[:,0] = 1
  
    # uh oh

    psi_dx = lambda inp: torch.autograd.functional.jvp(model, inp, v=e_x, create_graph=True)[1]
    psi_d2x = torch.autograd.functional.jvp(psi_dx, in_put, v=e_x, create_graph=True)[1]
    
    psi_d2x_real = psi_d2x[:,0]
    psi_d2x_imag = psi_d2x[:,1]
    
    # Calculate time derivative
    t = torch.zeros((batch_size,302)).to(device)
    t[:,1] = 1
    
#     psi_dt_real = torch.autograd.grad(psi_real, t, create_graph=True)
#     psi_dt_imag = torch.autograd.grad(psi_imag, t, create_graph=True)
    
    psi_dt = torch.autograd.functional.jvp(model, in_put, v=t, create_graph=True)[1]
    psi_dt_real = psi_dt[:,0]
    psi_dt_imag = psi_dt[:,1]
    
    # Calculate potential energy
    V_grid = in_put[:,2*grid_size+1:]
    pos = (in_put[:,0]*(grid_size-1)).type(torch.IntTensor).to(device)
    #V = np.choose(pos, V_grid.T)
    V = torch.index_select(V_grid,1,pos)
    
    V_real = V * psi_real
    V_imag = V * psi_imag
    
    # DEBUG
#     plt.figure()
#     xs = np.linspace(0,1,grid_size)
#     plt.plot(xs,psi_dt_real.detach().cpu().numpy()[0,:])
#     plt.plot(xs, (-0.5*d2_psi_imag + V_imag).detach().cpu().numpy()[0,:])
#     plt.show()
    
    # Calculate loss
    diff_1 = psi_dt_real + 0.5*psi_d2x_real - V_imag
    diff_2 = psi_dt_imag - 0.5*psi_d2x_imag + V_real
    
    return torch.sum(diff_1**2 + diff_2**2)


def custom_loss(output, in_put, target):
    mse =  F.mse_loss(output,y)
    #n_factor = norm_factor(output)
    schrod_loss = schrodinger_loss(output, in_put)
    
    return mse + schrod_hyperparam*schrod_loss
    

for epoch in range(nepochs):
    epoch_loss = 0
    
    for x,y in tqdm(train_data_loader):
        x = x.to(device)
        y = y.to(device)
        
        optm.zero_grad()
        output = model(x)
        #loss = F.mse_loss(output,y)        
        loss = custom_loss(output,x,y)
        
        loss.backward()
        optm.step()
        
        epoch_loss+=loss/len(train_data_loader)
        
    print('Epoch {} Loss : {:.3e}'.format((epoch+1),epoch_loss))

100%|██████████| 1000/1000 [00:21<00:00, 47.02it/s]


Epoch 1 Loss : 2.057e-01


100%|██████████| 1000/1000 [00:19<00:00, 52.47it/s]


Epoch 2 Loss : 1.917e-02


100%|██████████| 1000/1000 [00:19<00:00, 52.43it/s]


...

Epoch 22 Loss : 2.683e-03


100%|██████████| 1000/1000 [00:17<00:00, 57.23it/s]


Epoch 23 Loss : 2.652e-03


 70%|███████   | 703/1000 [00:13<00:05, 53.14it/s]


KeyboardInterrupt: 

In [ ]:
dt = 0.01
T = 0.5
ts = np.arange(0,T,dt)

grid_length = 1
grid_size = model.grid_size

import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

def solve_single_numerically(psi0, v, ts, grid_size=grid_size, grid_length=grid_length):
    xs_num = np.linspace(0,grid_length,grid_size)
    p0_real, p0_imag = psi0(xs_num)
    vs = v(xs_num)
    
    initials = np.zeros((3, grid_size, 1))
    initials[0, :, 0] = p0_real.T
    initials[1, :, 0] = p0_imag.T
    initials[2, :, 0] = vs.T
    
    num_y = numerical_schrodinger(initials, ts, grid_size=grid_size)
    
    num_ys_real = num_y[0,:,0,:]
    num_ys_imag = num_y[1,:,0,:]
    
    return num_ys_real.T, num_ys_imag.T


def solve_single_nn(psi0, v, ts, grid_size=grid_size, grid_length=grid_length):
    xs = torch.linspace(0, grid_length, grid_size).float()
    ts = torch.tensor(ts).float()
    p0_real, p0_imag = psi0(xs)
    vs = v(xs)
    
    if not torch.is_tensor(ts):
        ts = torch.tensor(ts)
        
    xts = torch.cartesian_prod(xs,ts)
#     xs_meshed = xts[:,0]
#     ts_meshed = xts[:,1]
    
    nn_in = torch.zeros((len(xts), 3*grid_size + 2))
    nn_in[:,0:2] = xts
    nn_in[:,2:] = torch.cat((p0_real, p0_imag, vs))
    nn_in = nn_in.to(device)
    
    nn_out = model(nn_in).cpu().detach().numpy()
    
    out_real = np.reshape(nn_out[:,0], (grid_size, len(ts))).T
    out_imag = np.reshape(nn_out[:,1], (grid_size, len(ts))).T 
    
    return out_real, out_imag
    

def test_model(psi0, v, plot_phase=False):
    # Solve numerically
    print('Solving numerically...')
    num_ys_real,num_ys_imag = solve_single_numerically(psi0, v, ts, grid_size, grid_length)
    print('Finished solving numerically.')
   
    # Solve using our method
    print('Solving nn...')
    nn_ys_real, nn_ys_imag = solve_single_nn(psi0, v, ts, grid_size, grid_length)
    print('Finished solving nn.')
    
    xs = np.linspace(0, grid_length, grid_size)
    
    # Normalisation vs time
    fig = plt.figure(figsize=(5,5))
    plt.plot(ts, np.sum(nn_ys_real**2 + nn_ys_imag**2, axis=1)*grid_length / grid_size)
    fig.suptitle('Normalisation')
    plt.xlabel('Time')
    plt.ylabel('∫|Ψ|² dx')
    
    # Plot animations
    fig = plt.figure(figsize=(12,8))
    
    plt.rcParams["animation.html"] = "html5"
    plt.rcParams["figure.dpi"] = 75
    
    # Helper for setting up subplot limits and labels
    def setup_subplot(subplot, prop):
        prop = prop.upper()
        if not prop in ["REAL", "IMAG", "ABS", "PHASE"]:
            raise ValueError(f'Bad property \'{prop}\'.')
        subplot.set_xlim(0,grid_length)
        if prop in ["REAL", "IMAG"]:
            subplot.set_ylim(-2,2)
            subplot.set_ylabel("Real" if prop == "REAL" else "Imaginary")
        if prop == "ABS":
            subplot.set_ylim(0,2)
            subplot.set_ylabel("Magnitude")
        if prop == "PHASE":
            subplot.set_ylim(-np.pi,np.pi)
            subplot.set_ylabel("Phase")
            subplot.set_yticks(np.arange(-np.pi,np.pi,np.pi/4))
        
        line, = subplot.plot([],[], lw=2)
        return line
    
    # Types of each subplot
    props = [None]*4
    if plot_phase:
        props = ["ABS", "ABS", "PHASE", "PHASE"]
    else:
        props = ["REAL", "REAL", "IMAG", "IMAG"]
       
    subplots = [None]*4
    lines = [None]*4
    for i in range(4):
        subplots[i] = plt.subplot(2,2,i+1)
        lines[i] = setup_subplot(subplots[i], props[i])
        
    subplots[0].title.set_text('NN model')
    subplots[1].title.set_text('Numerical model')
    
    def animate(i):
        if plot_phase:
            lines[0].set_data(xs, np.sqrt(nn_ys_real[i,:]**2 + nn_ys_imag[i,:]**2))
            lines[2].set_data(xs, np.arctan2(nn_ys_real[i,:], nn_ys_imag[i,:]))
        
            lines[1].set_data(xs, np.sqrt(num_ys_real[i,:]**2 + num_ys_imag[i,:]**2))
            lines[3].set_data(xs, np.arctan2(num_ys_real[i,:], num_ys_imag[i,:]))

        else:
            lines[0].set_data(xs, nn_ys_real[i,:])
            lines[2].set_data(xs, nn_ys_imag[i,:])
            
            lines[1].set_data(xs, num_ys_real[i,:])
            lines[3].set_data(xs, num_ys_imag[i,:])

        return lines
    
    anim = animation.FuncAnimation(fig, animate, frames=len(ts), interval=50, blit=True)
    HTML(anim.to_html5_video())
    return anim

In [ ]:
# Particle in a Box Ground State - Real+Imag

def psi0(x):
    real = np.sqrt(2)*np.sin(np.pi*x)
    imag = 0*x
    return real, imag

def v0(x):
    return 0*x

test_model(psi0, v0, plot_phase=False)

In [ ]:
# Particle in a Box Ground State - Amplitude+Phase

def psi0(x):
    real = np.sqrt(2)*np.sin(np.pi*x)
    imag = 0*x
    return real, imag

def v0(x):
    return 0*x

test_model(psi0, v0, plot_phase=True)

In [ ]:
# Particle in a Box - First Excited State

def psi0(x):
    real = np.sqrt(2)*np.sin(2*np.pi*x)
    imag = 0*x
    return real, imag

def v0(x):
    return 0*x

test_model(psi0, v0, plot_phase=False)

In [ ]:
import random
import matplotlib.pyplot as plt
def check_training_data(d,n=10):
    subset = random.sample(d,n)
    
    for s in subset:
        grid_size = int((len(s[0])-1)/3)
                
        init_real = s[0][1:grid_size+1]
        init_imag = s[0][grid_size+1:2*grid_size+1]
        
        final_real = s[1][:grid_size]
        final_imag = s[1][grid_size:]
        
        xs = np.linspace(0,1,grid_size)
                
        plt.figure()
        sbp1 = plt.subplot(2,2,1)
        sbp1.title.set_text('t=0')
        sbp1.set_xlim(0,1)
        sbp1.set_ylim(0,2)
        l1, = sbp1.plot(xs,np.sqrt(init_real**2 + init_imag**2), lw=2)

        sbp3 = plt.subplot(2,2,3)
        sbp3.set_xlim(0,1)
        sbp3.set_ylim(-1*np.pi,np.pi)
        l3, = sbp3.plot(xs,np.arctan2(init_real,init_imag), lw=2)

        sbp2 = plt.subplot(2,2,2)
        sbp2.title.set_text(f't={s[0][0]}')
        sbp2.set_xlim(0,1)
        sbp2.set_ylim(0,2)
        l2, = sbp2.plot(xs,np.sqrt(final_real**2 + final_imag**2), lw=2)

        sbp4 = plt.subplot(2,2,4)
        sbp4.set_xlim(0,1)
        sbp4.set_ylim(-1*np.pi,np.pi)
        l4, = sbp4.plot(xs, np.arctan2(final_real, final_imag), lw=2)
        

In [ ]:
check_training_data(data.data)

In [ ]:
# TEST FOR DEBUG

# Try to train
from tqdm import tqdm
import matplotlib.pyplot as plt # DEBUG TMP
import scipy.interpolate

nepochs = 100

train_data_loader = torch.utils.data.DataLoader(data, batch_size=100, shuffle=True)


mse_hyperparam = 1
norm_hyperparam = 0
schrod_hyperparam = 0

def norm_factor(output):
    norm = torch.sum(output**2)* data.grid_length / (np.prod(output.shape) / 2)
    norm_factor = (norm-1)**2
    return norm_factor

grid_size = model.grid_size
def schrodinger_loss(output, in_put):
    batch_size = output.shape[0]
    
    psi_real = output[:,:grid_size]
    psi_imag = output[:,grid_size:]
    
    # Calculate Laplacian
    xs = np.linspace(0,1,grid_size)
    psi_real_np = psi_real.detach().cpu().numpy()
    psi_imag_np = psi_imag.detach().cpu().numpy()
    
    print(xs.shape)
    print(psi_real_np.shape)
    
    d2_psi_real = scipy.signal.savgol_filter(psi_real_np,9,3,deriv=2, delta=1/100, axis=1)
    d2_psi_imag = scipy.signal.savgol_filter(psi_imag_np,9,3,deriv=2, delta=1/100, axis=1)
    
#     zero = torch.zeros((output.shape[0], 1)).to(device)
#     psi_real_exp = torch.cat((zero, psi_real, zero), dim=1)
#     psi_imag_exp = torch.cat((zero, psi_imag, zero), dim=1)
    
#     dx = data.grid_length / grid_size
#     d2_psi_real = (psi_real_exp[:,0:-2] + psi_real_exp[:,2:] - 2*psi_real_exp[:,1:-1]) / dx**2
#     d2_psi_imag = (psi_imag_exp[:,0:-2] + psi_imag_exp[:,2:] - 2*psi_imag_exp[:,1:-1]) / dx**2
    
    # Calculate time derivative
    t = torch.zeros((batch_size,301)).to(device)
    t[:,0] = 1
    
    psi_dt = torch.autograd.functional.jvp(model, in_put, v=t, create_graph=True)[1]
    print(psi_dt.shape)
    psi_dt_real = psi_dt[:,:grid_size]
    psi_dt_imag = psi_dt[:,grid_size:]
    
    # Calculate potential energy
    V = in_put[:,2*grid_size+1:]
    V_real = V * psi_real
    V_imag = V * psi_imag
    
    # DEBUG
    plt.figure()
    xs = np.linspace(0,1,grid_size)
    
    # Plot time derivative
    
#     dpt_1 = psi_dt_real.detach().cpu().numpy()[0,:]
#     dpt_2 = psi_dt_imag.detach().cpu().numpy()[0,:]
    
#     plt.plot(xs,np.sqrt(dpt_1**2 + dpt_1**2))
#     plt.show()
    
    px_1 = (-d2_psi_real)[0,:]
    px_2 = np.pi**2 * psi_real[0,:].detach().cpu().numpy()
    
    # Plot original wavefn
    plt.plot(xs, px_2)
    #plt.plot(xs,scipy.signal.savgol_filter(px_2,13,3), lw=1.25)
    plt.show()
    
    # Plot second dervative against original wavefn
    plt.plot(xs,px_1)
    plt.plot(xs,px_2)
    plt.show()


def custom_loss(output, in_put, target):
    mse =  F.mse_loss(output,y)
    n_factor = norm_factor(output)
    schrod_loss = schrodinger_loss(output, in_put)
    
    return mse + norm_hyperparam*n_factor + schrod_hyperparam*schrod_loss
    

for epoch in range(nepochs):
    epoch_loss = 0
    
    for x,y in tqdm(train_data_loader):
        x = x.to(device)
        y = y.to(device)

        output = model(x)      
        loss = custom_loss(output,x,y)

In [ ]:
# Test of B-spline and savgol smoothing and differentiation

import scipy.interpolate
import scipy.signal

n = 1

xs = np.linspace(0,1,100)
ys = np.sin(n*np.pi*xs) + np.random.rand(100)*0.025

ys_smooth = scipy.signal.savgol_filter(ys,13,3,deriv=0)
savgol_d2ydx2 = scipy.signal.savgol_filter(ys,13,3,deriv=2, delta=1/100)

tck = scipy.interpolate.splrep(xs,ys, k=5, s=0.01, task=0)
tck2 = scipy.interpolate.splder(tck,2)
bsplines_d2ydx2 = scipy.interpolate.splev(xs,tck2)


# plt.plot(xs,ys)
# plt.show()

plt.plot(xs,ys)
plt.title('Original function')
plt.show()

plt.plot(xs, -1*n**2*np.pi**2 * ys)
plt.plot(xs,savgol_d2ydx2)
plt.title('Savgol Filter 2nd Derivative')
plt.show()

plt.plot(xs, -1*n**2*np.pi**2 * ys)
plt.plot(xs,bsplines_d2ydx2)
plt.title('B Splines 2nd Derivative')
plt.show()


In [ ]:
len(data)